## Demographics, Severity (단일변수) 붙이기
- Demographics:
    - age (hadm id)
    - height
    - weight (admit, daily 어떻게 쓸지 결정 필요)
    - gender
- Severity:
    - Charlson, OASIS, APSIII, SAPSII

In [3]:
import numpy as np
import pandas as pd
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from datetime import timedelta
import time
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import sys
from sshtunnel import SSHTunnelForwarder
import logging
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')

# 데이터 정제 시 필요한 모듈들
import src.subjectlist_alignment.pairing as pairing   # 삽관 발관 페어링 관련 모듈
from src.subjectlist_alignment import reintubation as rnt   # 재삽관 시간 계산 관련 모듈
from src.subjectlist_alignment import imputation as imp   # 결측치 대체 관련 모듈
from src.subjectlist_alignment import subject_classification as cls

import src.utils.utils as utils   # 기타 유틸리티 모듈
import src.data_extraction.access_database as db   # DB 연결 관련 모듈

### subjectlist 가져오기

In [2]:
subjectlist1 = pd.read_csv('../outputs/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')
sub1list = sub1list.drop_duplicates(subset='stay_id')   # remove dup
sub1list

,subject_id,hadm_id,stay_id
0,10001884,26184834,37510196
1,10003400,23559586,38383343
2,10004401,27939719,31202136
3,10004401,29988601,32773003
5,10005817,28661809,31316840
...,...,...,...
8995,19995595,21784060,34670930
8997,19997367,20617667,35616526
9000,19999068,21606769,30143796
9001,19999442,26785317,32336619


### DB 연결 후 테이블 가져오기

In [2]:
# 1. 파라미터 설정

# 데이터 저장위치
# output_dir = './data'   # 수정 쌤 디렉토리에서 사용
output_dir = '../outputs'   # 내 디렉에서 사용


# SSH 연결 설정
ssh_config = {
    'ssh_ip': '210.91.223.248',
    'ssh_port': 35430,
    'ssh_username': "medicalai",
    'ssh_password': "medicalai731!",
    'remote_bind_address': ('localhost', 35432)  # 데이터베이스 호스트 및 포트
}

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_admin',
    'password': 'admin1q2w!@',
    'host': 'localhost',  # SSH 터널을 사용하면, 이 값은 'localhost'가 됩니다
    'port': 35432  # 이 값은 SSH 터널 로컬 포트 바인딩에 의해 대체됩니다
}


# 테이블 쿼리 설정
tables_query = {
    'patients': "select * from mimiciv_hosp.patients;",   # 환자 정보 (gender 여기서 추출)
    'age': "select * from mimiciv_derived.age;",   # 나이
    'height': "select * from mimiciv_derived.height;",   # 키
    'weight': "select * from mimiciv_derived.weight_durations;",   # 체중  (admit, daily)
    'charlson': "select * from mimiciv_derived.charlson;",   # CCI
    'oasis': "select * from mimiciv_derived.oasis;",   # OASIS
    'apsiii': "select * from mimiciv_derived.apsiii;",   # APS III
    'sapsii': "select * from mimiciv_derived.sapsii;"   # SAPS II
}


db.print_config_info(db_config, tables_query)

--------- Database Configuration ---------
Database Name: mimiciv
User: mai_admin
Password: ***********
Host: localhost
Port: 35432

--------- SQL Queries for Required Tables ---------
patients: select * from mimiciv_hosp.patients;
age: select * from mimiciv_derived.age;
height: select * from mimiciv_derived.height;
weight: select * from mimiciv_derived.weight_durations;
charlson: select * from mimiciv_derived.charlson;
oasis: select * from mimiciv_derived.oasis;
apsiii: select * from mimiciv_derived.apsiii;
sapsii: select * from mimiciv_derived.sapsii;



In [ ]:
# 2. db 접속, 테이블 추출, 'dataframes' 라는 딕셔너리에 테이블 저장
try:
    conn, tunnel = db.connect_to_database_via_ssh(db_config, ssh_config)
    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")
except Exception as e:
    logging.error(f"An error occurred: {e}")
finally:
    if conn is not None:
        db.disconnect_database(conn)


# 데이터프레임 변수로 저장
## Demographics
patients = dataframes['patients']
age = dataframes['age']

charlson = dataframes['charlson']
oasis = dataframes['oasis']
apsiii = dataframes['apsiii']
sapsii = dataframes['sapsii']

print(f'patients: {patients.shape}')
print(f'admissions: {admissions.shape}')
print(f'intubation_all: {intubation_all.shape}')
print(f'extubation_all: {extubation_all.shape}')
print(f'icustays: {icustays.shape}')

In [ ]:
# DB 연결 테이블 후 가져오기

database_config = {'database': 'mimic4',
 'user': 'postgres',
 'password': '1234',
 'host': 'localhost',
 'port': 5432}

tables_query = {
    'patients': "select * from mimiciv_hosp.patients;",   # 환자 정보 (gender 여기서 추출)
    'age': "select * from mimiciv_derived.age;",   # 나이
    'height': "select * from mimiciv_derived.height;",   # 키
    'weight': "select * from mimiciv_derived.weight_durations;",   # 체중  (admit, daily)
    'charlson': "select * from mimiciv_derived.charlson;",   # CCI
    'oasis': "select * from mimiciv_derived.oasis;",   # OASIS
    'apsiii': "select * from mimiciv_derived.apsiii;",   # APS III
    'sapsii': "select * from mimiciv_derived.sapsii;"   # SAPS II
                }


db.print_config_info(database_config, tables_query)
conn = db.connect_to_database(database_config)
dataframes = db.retrieve_data(conn, tables_query)
db.disconnect_database(conn)

patients = dataframes['patients']

age = dataframes['age']
charlson = dataframes['charlson']
oasis = dataframes['oasis']
apsiii = dataframes['apsiii']
sapsii = dataframes['sapsii']
